In [ ]:
import arcpy
import re
from netCDF4 import Dataset
import datetime
path_wordspace = "D:\\nc\\"
path_temp = "D:\\nc\\temp\\"
D = ".tif"
out_rasterdataset = "temp1"
files = open("D:\\nc\\nc.txt",'r')
for files_each_line in files:
    #ISCCP_HXG_global_radiation_1984_01_monthly
    files_each_line = files_each_line.replace('\n', '')
    year = re.findall("_\d\d\d\d_",files_each_line)
    year = str(year).replace("_']",'')
    year = str(year).replace("['_", '')
    mon = re.findall("_\d\d_", files_each_line)
    mon = str(mon).replace("_']", '')
    mon = str(mon).replace("['_", '')
    files_no_target = path_wordspace + str(year) + "/" + files_each_line + ".nc"
    files_no_temp = path_temp
    files_no_final = path_wordspace + "R\\mon\\"
    nc = files_no_target
    try:
        arcpy.MakeNetCDFRasterLayer_md(in_netCDF_file=nc, variable="monthly_global_radiation",
                                       x_dimension="lon", y_dimension="lat",
                                       out_raster_layer=str(files_each_line) + str(year) + str(mon),
                                       band_dimension="", dimension_values="",
                                       value_selection_method="BY_VALUE")
    except Exception as e:
        print(e)
    try:
        arcpy.CopyRaster_management(in_raster=str(files_each_line) + str(year) + str(mon),
                                    out_rasterdataset=files_no_temp + str(year) + str(mon) + D,
                                    config_keyword="", background_value="",
                                    nodata_value="-3.402823e+038",
                                    onebit_to_eightbit="NONE", colormap_to_RGB="NONE",
                                    pixel_type="",
                                    scale_pixel_value="NONE", RGB_to_Colormap="NONE", format="",
                                    transform="NONE")
        arcpy.Delete_management(str(files_each_line) + str(year) + str(mon))
    except Exception as e:
        print(e)
    try:
        arcpy.Resample_management(
            in_raster=files_no_temp + str(year) + str(mon) + D,
            out_raster=files_no_temp + "resample" + str(year) + str(mon) + D,
            cell_size="0.15 0.15",
            resampling_type="BILINEAR")
        arcpy.Delete_management(files_no_temp + str(year) + str(mon) + D)
    except Exception as e:
        print(e)
    try:
        arcpy.Clip_management(
            in_raster=files_no_temp + "resample" + str(year) + str(mon) + D,
            rectangle="95.8334536376953 32.1461541503906 118.096153637695 41.9296541503906",
            out_raster=files_no_final + "R" + str(year) + "-" + str(mon) + D,
            in_template_dataset="G:/zhutiansheng/CLIMATE/yellowriver.shp",
            nodata_value="-3.400000e+038", clipping_geometry="ClippingGeometry",
            maintain_clipping_extent="NO_MAINTAIN_EXTENT")
        print('success')
        arcpy.Delete_management(
            files_no_temp + "resample" + str(year) + str(mon) + D)
    except Exception as e:
        print(e)

success
